In [ ]:
!CMAKE_ARGS="-DLLAMA_CUBLAS=on" pip install llama-cpp-python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.7/10.7 MB 67.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for llama-cpp-python: filename=llama_cpp_python-0.2.38-cp310-cp310-manylinux_2_35_x86_64.whl size=9660822 sha256=897336c2591b1eeeae9299a370bdba7113a41c3e0fd66bbe72f6bafa36ca59d4
  Stored in directory: /root/.cache/pip/wheels/eb/58/77/20d3d9a235b4930050fbcde1ad4f0a4d054644269e801b08aa
Successfully built llama-cpp-python


In [ ]:
!pip install huggingface-hub

In [ ]:
!huggingface-cli download TheBloke/CapybaraHermes-2.5-Mistral-7B-GGUF capybarahermes-2.5-mistral-7b.Q4_K_M.gguf --local-dir . --local-dir-use-symlinks False

Consider using `hf_transfer` for faster downloads. This solution comes with some limitations. See https://huggingface.co/docs/huggingface_hub/hf_transfer for more details.
downloading https://huggingface.co/TheBloke/CapybaraHermes-2.5-Mistral-7B-GGUF/resolve/main/capybarahermes-2.5-mistral-7b.Q4_K_M.gguf to /root/.cache/huggingface/hub/tmp4mpy3nt1
(…)apybarahermes-2.5-mistral-7b.Q4_K_M.gguf: 100% 4.37G/4.37G [03:19<00:00, 21.9MB/s]
./capybarahermes-2.5-mistral-7b.Q4_K_M.gguf


In [ ]:
def generate_sentences_and_storyboard(context):
  from llama_cpp import Llama

  # Set gpu_layers to the number of layers to offload to GPU. Set to 0 if no GPU acceleration is available on your system.
  llm = Llama(
    model_path="./capybarahermes-2.5-mistral-7b.Q4_K_M.gguf",  # Download the model file first
    n_ctx=32768,  # The max sequence length to use - note that longer sequence lengths require much more resources
    n_threads=8,            # The number of CPU threads to use, tailor to your system and the resulting performance
    n_gpu_layers=35         # The number of layers to offload to GPU, if you have GPU acceleration available
  )
  # Simple inference example
  system_message="you are now a marketing designer for a video to promote a given policy. Come up with scenes and describe each scene with singaporean context and simple english and clearly on the shot also give content from the policy given to be narrated over. For example be specific Asian Family, Singaporean. Total minimum 25 scenes and 10 word max per sentence be clear and precise, the output format should strictly be Scene: Content: "
  prompt = context
  output = llm(
    f"<|im_start|>system\n{system_message}<|im_end|>\n<|im_start|>user\n{prompt}<|im_end|>\n<|im_start|>assistant", # Prompt
    max_tokens=1024,  # Generate up to 1024 tokens
    stop=["</s>"],   # Example stop token - not necessarily correct for this specific model! Please check before using.
    echo=False        # Whether to echo the prompt
)
  texts = output['choices'][0]['text']
  list = texts.split('\n')
  print(texts)
  storyboard = []
  clean_sentences = []
  narration = []
  for sentence in list:
    if 'Scene: ' in sentence:
      clean_sentences.append(sentence.split('Scene: ')[1])
      storyboard.append(sentence)
    elif 'Content: ' in sentence:
      narration.append(sentence.split('Content: ')[1])
      storyboard.append(sentence)

  return clean_sentences, storyboard, narration

In [ ]:
def generate_text_for_voiceover(data):
  from llama_cpp import Llama
  no_of_sentences=len(data)
  # Concatenated the sentences:
  combined_sentences=''
  for sentence in data:
    combined_sentences+= sentence + '\n'

  # Set gpu_layers to the number of layers to offload to GPU. Set to 0 if no GPU acceleration is available on your system.
  llm = Llama(
    model_path="./capybarahermes-2.5-mistral-7b.Q4_K_M.gguf",  # Download the model file first
    n_ctx=32768,  # The max sequence length to use - note that longer sequence lengths require much more resources
    n_threads=8,            # The number of CPU threads to use, tailor to your system and the resulting performance
    n_gpu_layers=35         # The number of layers to offload to GPU, if you have GPU acceleration available
  )
  # Simple inference example
  system_message=f"You are required to make a script for narration based on the context given without changing the facts, the output will be a paragraph that is {no_of_sentences*10} words speech do not exceed the word limit."
  prompt = combined_sentences
  output = llm(
    f"<|im_start|>system\n{system_message}<|im_end|>\n<|im_start|>user\n{prompt}<|im_end|>\n<|im_start|>assistant", # Prompt
    max_tokens=512,  # Generate up to 512 tokens
    stop=["</s>"],   # Example stop token - not necessarily correct for this specific model! Please check before using.
    echo=False        # Whether to echo the prompt
)
  texts = output['choices'][0]['text']
  return texts

In [ ]:
def generate_title_and_tagline(context):
  from llama_cpp import Llama

  # Set gpu_layers to the number of layers to offload to GPU. Set to 0 if no GPU acceleration is available on your system.
  llm = Llama(
    model_path="./capybarahermes-2.5-mistral-7b.Q4_K_M.gguf",  # Download the model file first
    n_ctx=32768,  # The max sequence length to use - note that longer sequence lengths require much more resources
    n_threads=8,            # The number of CPU threads to use, tailor to your system and the resulting performance
    n_gpu_layers=35         # The number of layers to offload to GPU, if you have GPU acceleration available
  )
  # Simple inference example
  system_message="You are required to generate a title and a tagline based on the content given"
  prompt = context
  output = llm(
    f"<|im_start|>system\n{system_message}<|im_end|>\n<|im_start|>user\n{prompt}<|im_end|>\n<|im_start|>assistant", # Prompt
    max_tokens=512,  # Generate up to 512 tokens
    stop=["</s>"],   # Example stop token - not necessarily correct for this specific model! Please check before using.
    echo=False        # Whether to echo the prompt
)
  texts = output['choices'][0]['text']
  texts = texts.split('\n')
  for sentence in texts:
    if 'Title: ' in sentence:
      # Note that the title has ""
      title=sentence.split('Title: ')[1].replace('"','')
    elif 'Tagline: ' in sentence:
      tagline=sentence.split('Tagline: ')[1].replace('"','').rstrip('.')
  return tagline, title

In [ ]:
def generate_theme_for_music(context):
  from llama_cpp import Llama

  # Set gpu_layers to the number of layers to offload to GPU. Set to 0 if no GPU acceleration is available on your system.
  llm = Llama(
    model_path="./capybarahermes-2.5-mistral-7b.Q4_K_M.gguf",  # Download the model file first
    n_ctx=32768,  # The max sequence length to use - note that longer sequence lengths require much more resources
    n_threads=8,            # The number of CPU threads to use, tailor to your system and the resulting performance
    n_gpu_layers=35         # The number of layers to offload to GPU, if you have GPU acceleration available
  )
  # Simple inference example
  system_message="You are required to generate a music genre name fit for a background music over the content. The output will be the genre only."
  prompt = context
  output = llm(
    f"<|im_start|>system\n{system_message}<|im_end|>\n<|im_start|>user\n{prompt}<|im_end|>\n<|im_start|>assistant", # Prompt
    max_tokens=512,  # Generate up to 512 tokens
    stop=["</s>"],   # Example stop token - not necessarily correct for this specific model! Please check before using.
    echo=False        # Whether to echo the prompt
)
  texts = output['choices'][0]['text']
  texts = texts.split('\n')

  for text in texts:
    if text !='':
      genre= text
  return genre

In [ ]:
context = """Healthier SG is a major transformation of our healthcare system. We are shifting our emphasis from reactively caring for those who are sick, to proactively preventing individuals from falling ill. The key is for our population to reshape their health-seeking behaviours and lifestyles. To achieve this, we need to anchor Singapore residents with a family doctor, and foster community support for healthier lifestyles. Healthier SG will prevent or delay the deterioration of health, reduce the burden on loved ones, extend healthy life years and improve residents’ quality of life. It will also moderate the increase in healthcare expenditure over time. Since March 2022, we have consulted more than 6,000 residents, and extensively engaged stakeholders such as private General Practitioners (GPs), employers and community partners to design the key features of Healthier SG, as described in this White Paper.
Residents:
Individuals who visit a regular family doctor are generally healthier and have fewer visits to the emergency departments and hospitals. However, most doctor-patient relationships today are transactional and episodic, with residents visiting doctors to get medication for acute symptoms or medical certificates. Under Healthier SG, residents will be encouraged to choose and enrol with a family doctor. Residents will have some flexibility to change their family doctor to take into account changes in their life circumstances. The family doctor will serve as the first point-of-contact to holistically manage the residents’ health. To encourage residents to follow their health plan and to seek care with their enrolled family doctor, we will: • Fully subsidise nationally recommended screenings and vaccinations for Singapore Citizens; • Waive the requirement for residents to co-pay part of their bills in cash when using MediSave for chronic care management; • Introduce a new Community Health Assist Scheme (CHAS) drug subsidy tier for a whitelist of chronic disease management drugs so that their prices at private GP clinics are comparable with those at polyclinics; and • Leverage the health points system under the National Steps Challenge to offer rewards to residents for enrolling and completing their first consultation with their selected family doctor and for leading active and healthy lifestyles
Community partner:
Improving our health goes beyond a doctor’s visit. Family doctors will be able to make social prescriptions and encourage residents to adopt healthier lifestyles. Healthier SG will make it easier for residents to consciously connect to the wide range of activities provided by agencies such as Health Promotion Board, Agency for Integrated Care, People’s Association, Sport Singapore, and community partners. Additional support will be made available for seniors through our Eldercare Centres (ECs). By 2025, we can expect both the network and the service offerings of all ECs to be expanded and enhanced.
"""

In [ ]:
data, story, narration = generate_sentences_and_storyboard(context)

AVX = 1 | AVX_VNNI = 0 | AVX2 = 1 | AVX512 = 1 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 1 | SSE3 = 1 | SSSE3 = 1 | VSX = 0 | 
Model metadata: {'tokenizer.chat_template': "{% for message in messages %}{{'<|im_start|>' + message['role'] + '\n' + message['content'] + '<|im_end|>' + '\n'}}{% endfor %}{% if add_generation_prompt %}{{ '<|im_start|>assistant\n' }}{% endif %}", 'tokenizer.ggml.add_eos_token': 'false', 'tokenizer.ggml.padding_token_id': '0', 'tokenizer.ggml.unknown_token_id': '0', 'tokenizer.ggml.eos_token_id': '32000', 'general.architecture': 'llama', 'llama.rope.freq_base': '10000.000000', 'llama.context_length': '32768', 'general.name': 'argilla_capybarahermes-2.5-mistral-7b', 'tokenizer.ggml.add_bos_token': 'true', 'llama.embedding_length': '4096', 'llama.feed_forward_length': '14336', 'llama.attention.layer_norm_rms_epsilon': '0.000010', 'llama.rope.dimension_count': '128', 'tokenizer.ggml.bos_


Scene: Residents at a park exercising
Content: Regular exercise is important for maintaining health. Healthier SG encourages residents to lead an active lifestyle.

Scene: Residents eating healthy meals at home
Content: Under Healthier SG, residents are encouraged to make healthier food choices.

Scene: Residents visiting a family doctor's clinic
Content: Residents will enroll with a family doctor for holistic health management.

Scene: Residents receiving nationally recommended screenings and vaccinations
Content: Healthier SG fully subsidizes these services for Singapore Citizens.

Scene: Residents discussing health plans with their family doctor
Content: The family doctor serves as the first point-of-contact for residents' health needs.

Scene: Residents using MediSave for chronic care management without co-payments
Content: Healthier SG waives part of residents' cash payments when using MediSave.

Scene: Residents at a community health event
Content: Healthier SG helps residents c

In [ ]:
tagline, title = generate_title_and_tagline(context)
summary = generate_text_for_voiceover(narration)
genre=generate_theme_for_music(context)
string = f'{tagline}, with title "{title}" on it,Movie Poster, MoviePosterRedAF,,'

AVX = 1 | AVX_VNNI = 0 | AVX2 = 1 | AVX512 = 1 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 1 | SSE3 = 1 | SSSE3 = 1 | VSX = 0 | 
Model metadata: {'tokenizer.chat_template': "{% for message in messages %}{{'<|im_start|>' + message['role'] + '\n' + message['content'] + '<|im_end|>' + '\n'}}{% endfor %}{% if add_generation_prompt %}{{ '<|im_start|>assistant\n' }}{% endif %}", 'tokenizer.ggml.add_eos_token': 'false', 'tokenizer.ggml.padding_token_id': '0', 'tokenizer.ggml.unknown_token_id': '0', 'tokenizer.ggml.eos_token_id': '32000', 'general.architecture': 'llama', 'llama.rope.freq_base': '10000.000000', 'llama.context_length': '32768', 'general.name': 'argilla_capybarahermes-2.5-mistral-7b', 'tokenizer.ggml.add_bos_token': 'true', 'llama.embedding_length': '4096', 'llama.feed_forward_length': '14336', 'llama.attention.layer_norm_rms_epsilon': '0.000010', 'llama.rope.dimension_count': '128', 'tokenizer.ggml.bos_

In [ ]:
string

'Uniting residents, family doctors, and community partners for a healthier and more sustainable Singapore, with title "Empowering Healthier Lifestyles: Shaping Singapore\'s Future with Healthier SG" on it,Movie Poster, MoviePosterRedAF,,'

In [ ]:
narration

['Regular exercise is important for maintaining health. Healthier SG encourages residents to lead an active lifestyle.',
 'Under Healthier SG, residents are encouraged to make healthier food choices.',
 'Residents will enroll with a family doctor for holistic health management.',
 'Healthier SG fully subsidizes these services for Singapore Citizens.',
 "The family doctor serves as the first point-of-contact for residents' health needs.",
 "Healthier SG waives part of residents' cash payments when using MediSave.",
 'Healthier SG helps residents connect to various activities provided by community partners.',
 'The challenge offers rewards for following health plans and leading active lifestyles.',
 "Additional support is provided for seniors through Healthier SG's enhanced Eldercare Centres.",
 'Family doctors can recommend health-boosting community activities.',
 'Healthier SG makes it easier for residents to access health information and resources.',
 'Subsidies make chronic disease m

In [ ]:
summary

"\nRegular exercise plays an essential role in maintaining good health, and Healthier SG is a program designed to encourage residents of Singapore to lead an active lifestyle. Through Healthier SG, residents are encouraged to not only engage in regular physical activity but also to make healthier food choices as part of their overall wellness plan.\n\nOne key aspect of Healthier SG is the relationship between residents and their family doctor. This primary care physician serves as the first point of contact for residents' health needs and plays an integral role in holistic health management. To further support residents in their quest for better health, Healthier SG fully subsidizes these services for Singapore Citizens, making it more accessible and affordable.\n\nHealthier SG also supports residents by waiving part of their cash payments when using MediSave, enabling them to save money while investing in their health. In addition, the program helps residents connect with various comm

In [ ]:
data

['Residents at a park exercising',
 'Residents eating healthy meals at home',
 "Residents visiting a family doctor's clinic",
 'Residents receiving nationally recommended screenings and vaccinations',
 'Residents discussing health plans with their family doctor',
 'Residents using MediSave for chronic care management without co-payments',
 'Residents at a community health event',
 'Residents participating in National Steps Challenge',
 'Residents attending an Eldercare Centre workshop',
 'Residents receiving social prescriptions from their family doctor',
 'Residents at a health workshop led by Health Promotion Board',
 'Residents using Community Health Assist Scheme drug subsidy at a private GP clinic',
 'Residents celebrating improved health and quality of life under Healthier SG',
 'Residents attending a community sports event',
 'Residents discussing their health plans with family members',
 'Residents receiving assistance at an Eldercare Centre',
 'Residents at a polyclinic receiv

In [ ]:
genre

'Health-Centric Symphony'

In [ ]:
story

['Scene: Residents at a park exercising',
 'Content: Regular exercise is important for maintaining health. Healthier SG encourages residents to lead an active lifestyle.',
 'Scene: Residents eating healthy meals at home',
 'Content: Under Healthier SG, residents are encouraged to make healthier food choices.',
 "Scene: Residents visiting a family doctor's clinic",
 'Content: Residents will enroll with a family doctor for holistic health management.',
 'Scene: Residents receiving nationally recommended screenings and vaccinations',
 'Content: Healthier SG fully subsidizes these services for Singapore Citizens.',
 'Scene: Residents discussing health plans with their family doctor',
 "Content: The family doctor serves as the first point-of-contact for residents' health needs.",
 'Scene: Residents using MediSave for chronic care management without co-payments',
 "Content: Healthier SG waives part of residents' cash payments when using MediSave.",
 'Scene: Residents at a community health ev

In [ ]:
import pickle

In [ ]:
# Specify file paths for each variable
story_pickle_file = 'story_data.pickle'
genre_pickle_file = 'genre_data.pickle'
tagline_pickle_file = 'tagline_data.pickle'
summary_pickle_file = 'summary_data.pickle'
string_pickle_file = 'string_data.pickle'
data_pickle_file = 'data.pickle'

# Export each variable to its respective pickle file
with open(story_pickle_file, 'wb') as pickle_file:
    pickle.dump(story, pickle_file)

with open(genre_pickle_file, 'wb') as pickle_file:
    pickle.dump(genre, pickle_file)

with open(tagline_pickle_file, 'wb') as pickle_file:
    pickle.dump(tagline, pickle_file)

with open(summary_pickle_file, 'wb') as pickle_file:
    pickle.dump(summary, pickle_file)

with open(string_pickle_file, 'wb') as pickle_file:
    pickle.dump(string, pickle_file)

with open(data_pickle_file, 'wb') as pickle_file:
    pickle.dump(data, pickle_file)